In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
import string

spark = SparkSession \
            .builder \
            .appName("StructuredNetworkWordCount") \
            .getOrCreate()






In [2]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
         .readStream \
         .format("socket") \
         .option("host", "localhost") \
         .option("port",9999) \
         .load()


In [3]:
from pyspark.sql.functions import regexp_replace
# Split the lines into words
words = lines.select(
        explode(
split(lines.value, " ")
   ).alias("word")
)

 # Generate running word count
wordCounts = words.groupBy("word").count()

In [4]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
          .writeStream \
          .outputMode("complete") \
          .format("memory") \
          .queryName("consulta1") \
          .start()



In [ ]:
print(lines.isStreaming)

In [ ]:
# Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

#query.awaitTermination()


In [5]:
from IPython.display import display, clear_output
from time import sleep

while True:
    clear_output(wait=True)
    display(query.status)
    display(spark.sql('SELECT * FROM consulta1').show())
    sleep(1)

{'message': 'Waiting for data to arrive',
 'isDataAvailable': False,
 'isTriggerActive': False}

+--------+-----+
|    word|count|
+--------+-----+
|caracola|    1|
|  pirola|    1|
|      me|    1|
|   crece|    1|
|    hola|    1|
|      la|    1|
+--------+-----+



None

KeyboardInterrupt: 

In [ ]:

# display(query.status)
# display(spark.sql('SELECT * FROM consulta1').show())
print (query.status)
print (spark.sql('SELECT * FROM consulta1').show())

In [ ]:
from pyspark.sql.functions import to_json
kafka_output_topic = "salida"

kafka_output_config = {
    "kafka.bootstrap.servers": "kafka-1:9092",  # Coloca aquí los servidores de arranque de Kafka
    "topic": kafka_output_topic
}

query = wordCounts \
    .selectExpr("CAST(word AS STRING) AS key", "CAST(count AS STRING) AS value") \
    .writeStream \
    .format("kafka") \
    .outputMode("complete") \
    .options(**kafka_output_config) \
    .start()
